Trying to use Just Pandas in this file

In [ ]:
import numpy as np
import math
import sys
import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This Cell are for installation of Spark

In [ ]:
!pip install pyspark
# !pip install findspark
!pyspark --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=c6169c514c2ded6aca3ae2c856296e939c6391a955ab87d095e4537967455a30
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.2
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.17
Branch HEAD
Compiled by user liangchi on 2023-02-10T19:57:40Z
Revision 5103e00c4ce5fcc4264ca9c4df12295d42557af6
Url https://github.com/apache/spark
Type --help for mo

In [ ]:
# import findspark
# findspark.init()
# findspark.find()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext ,SparkConf
# spark = SparkSession.builder\
#         .master("local")\
#         .appName("Colab")\
#         .config('spark.ui.port', '4050')\
#         .enableHiveSupport() \
#         .getOrCreate()

# spark = SparkSession \
#     .builder \
#     .appName('CCDN') \
#     .master("local[*]") \
#     .config("spark.driver.memory", "12g") \
#     .getOrCreate()
#print the SparkSession variable.
# spark = SparkConf().setMaster("local[2]").setAppName("CCDN")
# sc = SparkContext.getOrCreate();

conf = pyspark.SparkConf().setAll([
                                        ('spark.executor.memory', '8g'),
                                        ('spark.driver.maxResultSize', '8g'),
                                        ('spark.driver.memory','12g'),
                                        ('spark.sql.execution.arrow.pyspark.enabled', 'true')
                                        ])
spark = SparkSession.builder.config(conf=conf).appName("CCD").getOrCreate()
sc = spark.sparkContext

spark

Start of working on Dataset

In [ ]:
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)
ps.set_option('compute.default_index_type', 'distributed-sequence')
ps.set_option("display.max_rows", 11)
import math
import sys


/usr/local/lib/python3.8/dist-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
# for tiny.data
# df = spark.read.options(delimiter='\t', header=False).csv("/content/drive/My Drive/Colab Notebooks/Datasets/tiny.data")
# df = df.toDF("userId","itemId","rating","timestamp")
# df = df.drop("timestamp")
# df.printSchema()
# df = df.to_pandas_on_spark()


# for u.data
df = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/u.data" , sep ="\t" , names = ["userId", "itemId", "rating", "timestamp"])
df = df.drop("timestamp", axis=1)


# df.printSchema()
# df.show(6)
print(type(df))
# print(df)
df.count()
# print(df)

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


<class 'pyspark.pandas.frame.DataFrame'>


userId    100000
itemId    100000
rating    100000
dtype: int64

In [ ]:
# Converting userId and itemId to integer
df[['userId', 'itemId', 'rating']] = df[['userId', 'itemId', 'rating']].astype(int)
print(type(df))
print(df.dtypes)
df = df.spark.local_checkpoint()

<class 'pyspark.pandas.frame.DataFrame'>
userId    int64
itemId    int64
rating    int64
dtype: object


In [ ]:
def randomMatrix(dimension1, dimenstion2, rank):
    return ps.DataFrame(abs(np.random.normal(scale=1./rank,size=(dimension1,dimenstion2))))

def get_matrix_dimension(df):
  # This function get max user)id and itam_id as reslult, V and H matrix dimension
  userMax = df['userId'].max()
  itemMax = df['itemId'].max()
  userMax += 1
  itemMax += 1
  return userMax, itemMax

def sub_V_WH(WH):
  WH['R'] = WH['rating'] - WH['WH']
  R = WH.drop(['rating', 'WH'], axis=1)
  R = R.spark.local_checkpoint()
  return R

def mult_WH(W, H, WH, rank):
  WH = WH.merge(W,on='userId').merge(H,on='itemId')
  # print(WH)
  colst = ''
  colList = []
  for k in range (rank):
    colst += f"WH['WC{k}'] * WH['HC{k}'] + "
    colList.append(f"WC{k}")
    colList.append(f"HC{k}")
  colst = colst[0:len(colst)-2]
  # print(colst)
  # print(colList)
  WH['WH'] = eval(colst)
  WH = WH.drop(colList, axis=1)
  WH = WH.spark.local_checkpoint()
  return WH

In [ ]:
# Split Dataframe to Train and Test
num_split = 100000
train_data = df.iloc[:num_split,:]
test_data = df.iloc[num_split:,:]
# print(train_data)
# print(test_data)
userMax, itemMax = get_matrix_dimension(train_data)
print(f"{userMax}, {itemMax}")

944, 1683


In [ ]:
def mult_Wk_Hk(W, H, WkHk, k):
  # print("mult_Wk_Hk started...")
  # query = f"select userId, itemId, rating, WC{k} * HC{k} as WH from WkHk, W, H where W.id = WkHk.userId and H.id = WkHk.itemId"
  WkHk = WkHk.merge(W[[f'WC{k}','userId']],on='userId').merge(H[[f'HC{k}','itemId']],on='itemId')
  # print(WkHk)
  WkHk['WkHk'] = WkHk[f'WC{k}'] * WkHk[f'HC{k}']
  WkHk = WkHk.drop([f'WC{k}', f'HC{k}'], axis=1)
  # print(WkHk)
  # print("mult_Wk_Hk ended!")
  return WkHk

def R_add_WkHk(R, WkHk):
  # print(R)
  # print(WkHk)
  WkHk = WkHk.merge(R,on=['userId', 'itemId'])
  WkHk['Rhat'] = WkHk['R'] + WkHk['WkHk']
  Rhat = WkHk.drop(['rating', 'WkHk', 'R'], axis=1)
  # print(Rhat)
  Rhat = Rhat.spark.local_checkpoint()
  return Rhat
  # Rhat = spark1.sql("SELECT int(R.userId), int(R.itemId), (float(R.R) + float(WkHk.WH)) as Rhat  FROM R, WkHk where R.itemId = WkHk.itemId and R.userId = WkHk.userId")

def Rhat_minus_WkHk(Rhat, WkHk):
  # print("Rhat_minus_WkHk started...")
  WkHk = WkHk.merge(Rhat,on=['userId', 'itemId'])
  WkHk['R'] = WkHk['Rhat'] - WkHk['WkHk']
  R = WkHk.drop(['rating', 'WkHk', 'Rhat'], axis=1)
  # print(R)
  R = R.spark.local_checkpoint()
  # print("Rhat_minus_WkHk ended!")
  return R
  # R = spark1.sql("SELECT int(Rhat.userId), int(Rhat.itemId), (float(Rhat.Rhat) - float(WkHk.WH)) as R FROM Rhat, WkHk where Rhat.itemId = WkHk.itemId and Rhat.userId = WkHk.userId")

def Update_W_H(W, H, Rhat, k, lamda):
  # print(f" k = {k}")
  # print(Rhat)
  # print("Update_W_H started...")
  # print("Start Select aggregation...")
  # query = f"SELECT Rhat.userId, sum(Rhat.Rhat * H.HC0) / (sum(power(H.HC0,2))+{lamda}) as value FROM Rhat, H where Rhat.itemId = H.id group by Rhat.userId"

  WkHk = Rhat.merge(H[[f'HC{k}','itemId']],on='itemId')
  WkHk['s'] = WkHk['Rhat'] * WkHk[f'HC{k}']
  WkHk['m'] = WkHk[f'HC{k}'] ** 2
  # print(WkHk)
  sums = WkHk.groupby('userId').agg({'s': 'sum'}).reset_index(level=0)
  summ = WkHk.groupby('userId').agg({'m': 'sum'}).reset_index(level=0)
  new_W_ik = sums.merge(summ,on='userId')
  new_W_ik[f'WC{k}'] = new_W_ik['s'] / (new_W_ik['m'] + lamda)
  new_W_ik = new_W_ik.drop(['s', 'm'], axis=1)
  new_W_ik = new_W_ik.spark.local_checkpoint()

  # query = f"SELECT Rhat.itemId, sum(Rhat.Rhat * W.WC0) / (sum(power(W.WC0,2))+{lamda}) as value FROM W join Rhat on W.id = Rhat.userId group by Rhat.itemId"
  WkHk = Rhat.merge(W[[f'WC{k}','userId']],on='userId')
  WkHk['s'] = WkHk['Rhat'] * WkHk[f'WC{k}']
  WkHk['m'] = WkHk[f'WC{k}'] ** 2
  # print(WkHk)
  sums = WkHk.groupby('itemId').agg({'s': 'sum'}).reset_index(level=0)
  summ = WkHk.groupby('itemId').agg({'m': 'sum'}).reset_index(level=0)
  new_H_ik = sums.merge(summ,on='itemId')
  new_H_ik[f'HC{k}'] = new_H_ik['s'] / (new_H_ik['m'] + lamda)
  new_H_ik = new_H_ik.drop(['s', 'm'], axis=1)
  new_H_ik = new_H_ik.spark.local_checkpoint()

  # print(W)
  W = W.drop(f'WC{k}', axis=1)
  W = W.merge(new_W_ik, on = 'userId')
  W = W.spark.local_checkpoint()
  # print(new_W_ik)
  # print(W)

  # print(H)
  H = H.drop(f'HC{k}', axis=1)
  H = H.merge(new_H_ik, on = 'itemId')
  H = H.spark.local_checkpoint()
  # print(new_H_ik)
  # print(H)
  return W, H

def ccd_iteration(W, H, R, k):
  for rank in range(k):
    WkHk = mult_Wk_Hk(W, H, V, rank)
    # print(WkHk)
    # print("WkHk calculated For Rhat.")
    Rhat = R_add_WkHk(R, WkHk)
    # print(Rhat)
    # print("Rhat calculated.")
    W, H = Update_W_H(W, H, Rhat, rank, lamda)

    # print(W)
    # print(H)
    # sys.exit()
    # print("W, H  Updated on k.")
    WkHk = mult_Wk_Hk(W, H, V, rank)
    # WkHk.show()
    # print("WkHk calculated for R.")
    R = Rhat_minus_WkHk(Rhat, WkHk)
    # R.show()
    # print("R calculated.")
  return W, H, R

def rmse_WH_V(WH):
  WH['diff'] = (WH['rating'] - WH['WH']) ** 2
  rmse = math.sqrt(WH['diff'].mean())
  return rmse


In [ ]:
# from numpy import genfromtxt
k=10
lamda=60
print('rank: \t',k,'')
print('lamda: \t',lamda,'\n')

print(f"{userMax}, {itemMax}")

W = randomMatrix(userMax, k, k)
H = randomMatrix(k, itemMax, k)
H = H.T

W = W.reset_index(level=0)
H = H.reset_index(level=0)
WcolDic = {"index": "userId"}
HcolDic = {"index": "itemId"}
for rank in range (k):
  WcolDic[rank] = f"WC{rank}"
  HcolDic[rank] = f"HC{rank}"
# print(WcolDic)
# print(HcolDic)
W = W.rename(columns=WcolDic, errors="raise")
H = H.rename(columns=HcolDic, errors="raise")

tiny=False
if tiny==True:
  # Save to disk
  # H.write.options(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Htiny.data")
  # W.write.options(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Wtiny.data")
  # Read from disk
  W = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/Wtiny.data" , sep ="\t")
  H = ps.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/Htiny.data" , sep ="\t")
  # H = spark.read.options(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Htiny.data")
  # W = spark.read.options(delimiter='\t', header=True).csv("/content/drive/My Drive/Colab Notebooks/Datasets/Wtiny.data")
  # W = W.to_pandas_on_spark()
  # H = H.to_pandas_on_spark()
  W = W.rename(columns={"id": "userId", "WC0": "WC0", "WC1": "WC1"}, errors="raise")
  H = H.rename(columns={"id": "itemId", "HC0": "HC0", "HC1": "HC1"}, errors="raise")
  H = H.astype({'itemId':'int', 'HC0':'float', 'HC1':'float' })
  W = W.astype({'userId':'int', 'WC0':'float', 'WC1':'float' })

# print(W)
# print(H)

V = train_data
V = V.spark.local_checkpoint()
# print(V)

WH = mult_WH(W, H, V, k)
WH = WH.spark.local_checkpoint()
# print(WH)
print(type(WH))

R = sub_V_WH(WH)
R = R.spark.local_checkpoint()
# print(R)

# print("start calculation of rmse... ")
rmse = rmse_WH_V(WH)
print(f"RMSE: {rmse}")
# print("calculation of rmse end... ")


rank: 	 10 
lamda: 	 60 

944, 1683
<class 'pyspark.pandas.frame.DataFrame'>
RMSE: 3.644480046540599


In [ ]:
# print(R)
# print(W)
# print(H)
# print(WH)
iteration = 10
for i in range(iteration):
  print(f"Iteration: {i}")
  print(datetime.datetime.now())

  W, H, R = ccd_iteration(W, H, R, k)
  # sys.exit()
  # print(W)
  # print(H)
  # print(R)
  if not (i+1)%1:
    # print("start calculation of new WH for compute rmse... ")
    WH = mult_WH(W, H, V, k)
    # print(WH)
    # print("calculation of new WH end... ")
    # print("start calculation of rmse... ")
    rmse = rmse_WH_V(WH)
    print(f"RMSE: {rmse}")
    # sys.exit()

    # print("calculation of rmse end... ")
    if rmse < 0.5:
      print("Convergenced!")
      break

Iteration: 0
2023-02-19 16:08:46.916876
RMSE: 2.2093256297135406
Iteration: 1
2023-02-19 16:10:19.643893
RMSE: 1.2092355372254284
Iteration: 2
2023-02-19 16:11:23.859547
RMSE: 1.2439692670873008
Iteration: 3
2023-02-19 16:12:20.720164
RMSE: 1.2495702032807594
Iteration: 4
2023-02-19 16:13:15.517508
RMSE: 1.2467468253619123
Iteration: 5
2023-02-19 16:14:05.001780
RMSE: 1.247621310348523
Iteration: 6
2023-02-19 16:14:56.002108
RMSE: 1.247382187651163
Iteration: 7
2023-02-19 16:15:45.670742
RMSE: 1.2476432033313853
Iteration: 8
2023-02-19 16:16:34.248005
RMSE: 1.2476815126907097
Iteration: 9
2023-02-19 16:17:23.388761
RMSE: 1.2477036927664629


In [ ]:
userMax, itemMax = get_matrix_dimension(WH)
print(f"{userMax}, {itemMax}")
# WH.count()
print(WH)
print(W)
print(H)


944, 1683
    userId  itemId  rating        WH      diff
0       26     258       3  3.225774  0.050974
1       26     930       2  1.738350  0.068461
2       26    1015       3  0.883779  4.478389
3       29     245       3  2.292197  0.500986
4       26     125       4  2.675507  1.754283
5       29     189       4  2.303345  2.878639
6       26     685       3  2.379525  0.384989
7       26     235       2  2.080135  0.006422
8       29     332       4  2.420119  2.496023
9       29     539       2  1.294670  0.497490
10      26      25       3  2.620272  0.144193

[Showing only the first 11 rows x 5 columns]
    userId       WC0       WC1       WC2       WC3       WC4       WC5       WC6       WC7       WC8       WC9
0      474  1.291672  0.995547  0.690028  0.624193  0.554745  0.449316  0.408958  0.423849  0.397280  0.310760
1       26  0.798834  0.689502  0.492871  0.418925  0.349405  0.327160  0.307116  0.264381  0.220589  0.246800
2       29  0.735918  0.619339  0.445678  0.381